In [11]:
import math
import datasets
import IBM1 as ibm1

# pretty print variabeles on line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Some helper functions to print useful information after each 
# EM iteration

def print_likelihood(i, _, log_likelihood):
    likelihood = math.exp(log_likelihood)
    if i == 1:
        print('iteration  log_likelihood  likelihood')
    print(f'{i} {log_likelihood:.3f} {likelihood:.3f}')

def print_lexicon_probs(_, lprobs, _):
    for s in lprobs.keys():
        for t in lprobs[s].keys():
            if lprobs[s][t] > 0:
                print (s, t, lprobs[s][t])

In [13]:
# Run EM on toy example
s_t_pairs, s_vocabulary, t_vocabulary = datasets.example_data()
s_t_pairs
lprobs = ibm1.EM(s_t_pairs, s_vocabulary, t_vocabulary, print_likelihood)
lprobs

[(['das', 'Haus'], ['the', 'house']),
 (['das', 'Buch'], ['the', 'book']),
 (['ein', 'Buch'], ['a', 'book'])]

iteration  log_likelihood  likelihood
1 -4.159 0.016
2 -1.151 0.316
3 -0.842 0.431
4 -0.586 0.557
5 -0.390 0.677
6 -0.252 0.777
7 -0.159 0.853
8 -0.100 0.905
9 -0.062 0.940
10 -0.039 0.961
11 -0.025 0.975
12 -0.017 0.983
13 -0.012 0.989
14 -0.008 0.992
15 -0.006 0.994
16 -0.005 0.995
17 -0.004 0.996
18 -0.003 0.997
19 -0.003 0.997
20 -0.002 0.998


defaultdict(<function IBM1._initialize_lexicon_probabilities.<locals>.<lambda>>,
            {'Buch': defaultdict(int,
                         {'a': 9.729458618407092e-06,
                          'book': 0.9999876443477214,
                          'house': 0.0,
                          'the': 2.6261936603029755e-06}),
             'Haus': defaultdict(int,
                         {'a': 0.0,
                          'book': 0.0,
                          'house': 0.9684579278295524,
                          'the': 0.031542072170447605}),
             'das': defaultdict(int,
                         {'a': 0.0,
                          'book': 2.6261936603029776e-06,
                          'house': 9.729458618407095e-06,
                          'the': 0.9999876443477212}),
             'ein': defaultdict(int,
                         {'a': 0.9684579278295524,
                          'book': 0.031542072170447605,
                          'house': 0.0,
                     

In [12]:
# Run EM on toy example with NULL words
s_t_pairs, s_vocabulary, t_vocabulary = datasets.example_data_null_words()
s_t_pairs
lprobs = ibm1.EM(s_t_pairs, s_vocabulary, t_vocabulary, print_likelihood)
lprobs

# QUESTION: how can likelihood be bigger than 1?
# QUESTION: these NULL behavior is hard to interpret

[(['<NULL>', 'das', 'Haus'], ['the', 'house']),
 (['<NULL>', 'das', 'kleine', 'Buch'], ['the', 'book']),
 (['<NULL>', 'ein', 'Buch'], ['a', 'book']),
 (['<NULL>', 'ein', 'Haus'], ['a', 'small', 'house'])]

iteration  log_likelihood  likelihood
1 -4.022 0.018
2 -0.679 0.507
3 -0.064 0.938
4 0.368 1.445
5 0.643 1.901
6 0.804 2.235
7 0.897 2.452
8 0.949 2.584
9 0.979 2.662
10 0.996 2.709
11 1.007 2.737
12 1.013 2.754
13 1.017 2.765
14 1.020 2.773
15 1.022 2.778
16 1.023 2.781
17 1.024 2.784
18 1.025 2.786
19 1.025 2.788
20 1.026 2.789


defaultdict(<function IBM1._initialize_lexicon_probabilities.<locals>.<lambda>>,
            {'<NULL>': defaultdict(int,
                         {'a': 0.40297069736408325,
                          'book': 0.0362274366767834,
                          'house': 0.40297069736408325,
                          'small': 0.12160373191826655,
                          'the': 0.0362274366767834}),
             'Buch': defaultdict(int,
                         {'a': 8.668374184480112e-06,
                          'book': 0.9999913173858771,
                          'house': 0.0,
                          'small': 0.0,
                          'the': 1.4239938457654701e-08}),
             'Haus': defaultdict(int,
                         {'a': 8.90343310153906e-07,
                          'book': 0.0,
                          'house': 0.768183624465758,
                          'small': 0.23181337041273017,
                          'the': 2.114778201611731e-06}),
             'das': def

In [ ]:
# Run EM on training data set
s_t_pairs, s_vocabulary, t_vocabulary = datasets.training_data(
    "training/hansards.36.2.f", "training/hansards.36.2.e")
#lprobs = ibm1.EM(s_t_pairs, s_vocabulary, t_vocabulary)